In [79]:
##helper libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from subprocess import check_output
from collections import Counter
import gc
#keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers.wrappers import Bidirectional
from keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D, SpatialDropout1D
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder
#model export
import time
from keras import metrics
import h5py
from keras.models import model_from_json

In [13]:
#import data
dat = pd.read_csv('D:\\Azim\\CNN\\train.txt',sep='\t',header=None,encoding='latin')
tags = dat[0].str[2:7]+dat[0].str[9]
texts = dat[0].str.replace(pat='__label__2',n=1,repl='')
texts = dat[0].str.replace(pat='__label__1',n=1,repl='')

#free memory
del dat
gc.collect()
dat=pd.DataFrame()

In [14]:
num_max = 1000
tok = Tokenizer(num_words=num_max)
tok.fit_on_texts(texts)

In [15]:
# preprocess
le = LabelEncoder()
tags = le.fit_transform(tags)

# for cnn preproces
max_len = 100
cnn_texts_seq = tok.texts_to_sequences(texts)
print(cnn_texts_seq[0])
cnn_texts_mat = sequence.pad_sequences(cnn_texts_seq,maxlen=max_len)
print(cnn_texts_mat[0])
print(cnn_texts_mat.shape)

[23, 16, 74, 11, 1, 615, 8, 176, 490, 13, 367, 6, 1, 10, 60, 442, 28, 69, 3, 40, 6, 74, 5, 135, 72, 684, 145, 119, 3, 21, 529, 1, 145, 17, 41, 7, 29, 7, 1, 601, 3, 21, 132, 529, 6, 45, 1, 88, 119, 6, 242, 38, 2, 427, 4, 863, 18, 2, 6, 40, 208, 72, 5, 339]
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
  23  16  74  11   1 615   8 176 490  13 367   6   1  10  60 442  28  69
   3  40   6  74   5 135  72 684 145 119   3  21 529   1 145  17  41   7
  29   7   1 601   3  21 132 529   6  45   1  88 119   6 242  38   2 427
   4 863  18   2   6  40 208  72   5 339]
(3600000, 100)


In [35]:
# Embedding
max_features = 20000
maxlen = 100
embedding_size = 128

# Convolution
kernel_size = 5
filters = 64
pool_size = 4

# LSTM
lstm_output_size = 70

# Training
batch_size = 30
epochs = 2

In [80]:
model = Sequential()
model.add(Embedding(max_features, embedding_size, input_length=maxlen))
#model.add(Dropout(0.2))
#model.add(Conv1D(filters,
#                 kernel_size,
#                 padding='valid',
#                 activation='relu',
#                 strides=1))
model.add(SpatialDropout1D(0.2))
model.add(Bidirectional(LSTM(256, dropout=0.2)))
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')

Train...


In [84]:
model.fit(cnn_texts_mat,tags,batch_size=32,epochs=3,verbose=1,validation_split=0.2)

Train on 2880000 samples, validate on 720000 samples
Epoch 1/3
2880000/2880000 [==============================] - 62896s 22ms/step - loss: 0.0579 - acc: 0.9748 - val_loss: 0.0461 - val_acc: 0.9808
Epoch 2/3
2880000/2880000 [==============================] - 62227s 22ms/step - loss: 0.0470 - acc: 0.9802 - val_loss: 0.0425 - val_acc: 0.9825
Epoch 3/3
2880000/2880000 [==============================] - 61540s 21ms/step - loss: 0.0464 - acc: 0.9806 - val_loss: 0.0425 - val_acc: 0.9823


In [85]:
#import data
dat = pd.read_csv('D:\\Azim\\CNN\\test.txt',sep='\t',header=None,encoding='latin')
test_tags = dat[0].str[2:7]+dat[0].str[9]
test_texts = dat[0].str.replace(pat='__label__2',n=1,repl='')
test_texts = dat[0].str.replace(pat='__label__1',n=1,repl='')

In [86]:
#free memory
del dat
gc.collect()
dat=pd.DataFrame()

In [87]:
num_max = 1000
test_tok = Tokenizer(num_words=num_max)
test_tok.fit_on_texts(test_texts)

In [88]:
# for cnn preproces
max_len = 100
test_cnn_texts_seq = test_tok.texts_to_sequences(test_texts)
print(test_cnn_texts_seq[0])
test_cnn_texts_mat = sequence.pad_sequences(test_cnn_texts_seq,maxlen=max_len)
print(test_cnn_texts_mat[0])
print(test_cnn_texts_mat.shape)

[23, 16, 31, 94, 21, 45, 26, 7, 1, 31, 7, 79, 3, 22, 5, 8, 94, 11, 137, 2, 3, 127, 81, 6, 52, 110, 10, 4, 33, 6, 210, 44, 233, 91, 4, 128, 38, 34, 10, 1, 8, 94, 38, 154, 25, 2, 651, 38, 26, 7, 8, 9, 4, 94, 10, 21, 20, 180, 97, 124, 129, 6, 224, 9, 38, 871, 44, 3, 239, 8, 53, 619, 425, 581, 467, 134, 512, 26, 163, 72, 13, 12, 925]
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0  23
  16  31  94  21  45  26   7   1  31   7  79   3  22   5   8  94  11 137
   2   3 127  81   6  52 110  10   4  33   6 210  44 233  91   4 128  38
  34  10   1   8  94  38 154  25   2 651  38  26   7   8   9   4  94  10
  21  20 180  97 124 129   6 224   9  38 871  44   3 239   8  53 619 425
 581 467 134 512  26 163  72  13  12 925]
(400000, 100)


In [89]:
# preprocess
le = LabelEncoder()
test_tags = le.fit_transform(test_tags)

In [90]:
print(test_tags.shape)

(400000,)


In [91]:
scores = model.evaluate(test_cnn_texts_mat,test_tags)

400000/400000 [==============================] - 2655s 7ms/step


In [92]:
print("Model accuracy on test data is",scores[1]*100,"percent")

Model accuracy on test data is 94.84825000000001 percent


In [93]:
model_json = model.to_json()

In [94]:
model_backup=model

In [95]:
with open("D:\\Azim\\LSTM_Bidirectional\\model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5

In [96]:
#import cythonmagic
model.save_weights("D:\\Azim\\LSTM_Bidirectional\\model.h5")
print("Saved model to disk")

Saved model to disk
